## Import library

In [ ]:
import time
import numpy as np
from tqdm.notebook import trange,tqdm
import math

## Use our collected trace

If you do not have a ChipWhisperer device, you can use the traces we collected and skip the following trace collecting part. It may take some time to load the trace.

Remember to replace the file path.

In [ ]:
your_path = "add_your_path_here"
trace_cwlite = np.loadtxt(your_path+"/data/cwlite_cross.txt",delimiter=',')
trace_cwnano = np.loadtxt(your_path+"/data/cwnano_cross.txt",delimiter=',')

## Collect trace (ChipWhisperer-Lite) 

Connect the ChipWhisperer-Lite to the PC

In [ ]:
import chipwhisperer as cw
## Collect trace (ChipWhisperer-NANO)SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

In [ ]:
n_trace_all = 20000

In [ ]:
ktp = cw.ktp.Basic()
trace_array = []
textin_array = []
text = []

for i in range(n_trace_all):
    key, text_t = ktp.next()
    text.append(text_t)

In [ ]:
def capture(key, text):
    target.set_key(key)
    
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
    
    response = target.simpleserial_read('r', 16)
    
    return scope.get_last_trace()

In [ ]:
# capture trace
trace_cwlite = []

scope.adc.samples = 7500

scope.adc.decimate = 4

for i in trange(n_trace_all):
    trace_cwlite.append(np.array(capture(key, text[i])))

In [ ]:
scope.dis()

## Collect trace (ChipWhisperer-NANO)

Connect the ChipWhisperer-Nano to the PC

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWNANO'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

In [ ]:
ktp = cw.ktp.Basic()
trace_array = []
textin_array = []
text = []

for i in range(n_trace_all):
    key, text_t = ktp.next()
    text.append(text_t)

In [ ]:
def capture(key, text):
    target.set_key(key)
    
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
    
    response = target.simpleserial_read('r', 16)
    
    return scope.get_last_trace()

In [ ]:
# capture trace
trace_cwnano = []

scope.adc.samples = 13000

for i in trange(n_trace_all):
    trace_cwnano.append(np.array(capture(key, text[i])))

In [ ]:
scope.dis()

## Data Processing

In [ ]:
def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X):
    X_bar = mean(X)
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def corr(X, Y):
    X_bar = mean(X)
    Y_bar = mean(Y)
    return (np.sum((X-X_bar)*(Y-Y_bar), axis=0)/(std_dev(X)*std_dev(Y)))

In [ ]:
n_trace = n_trace_all
n_trace_half = n_trace >> 1

# train_cwlite = trace_cwlite[0:n_trace_half]
# train_cwnano = trace_cwnano[0:n_trace_half]
train_cwlite = trace_cwlite[0:1000,150:7150]
train_cwnano = trace_cwnano[0:1000]

In [ ]:
test_cwlite = trace_cwlite[n_trace_half:n_trace,150:7150]
test_cwnano = trace_cwnano[n_trace_half:n_trace]

In [ ]:
nano_start = 0
nano_end = 11500
nano_len = nano_end - nano_start

train_cwnano_ref = train_cwnano[0][nano_start:nano_end]

In [ ]:
# select POIs of NANO by variance
train_cwnano_var = []
poi_nano_big = []

n_poi = 1 # k_poi
interval = 500 # l_s

pieces = math.floor(nano_len/interval)

for i in range(nano_start,nano_end):
    train_cwnano_var.append(np.var(train_cwnano[:,i]))

for i in range(pieces):
    poi_nano_idx = np.array(train_cwnano_var[i*interval:i*interval+interval]).argsort()[-n_poi:][::-1]

    for idx in poi_nano_idx:
        true_idx = nano_start + i*interval+idx
        poi_nano_big.append([true_idx,np.mean(train_cwnano[:,true_idx])])
    


In [ ]:
# select POIs of Lite corresponding to the POIs of NANO
poi_candi = []

for poi in tqdm(poi_nano_big):
    curr_poi_nano = poi

    v_points_nano = train_cwnano[:,curr_poi_nano[0]]

    features = []

    for i in range(len(train_cwlite[0])):
        v_points_lite = train_cwlite[:,i]
        cor = corr(v_points_nano,v_points_lite)
        features.append(cor)

    corr_poi = max(features,key=abs)
    idx = np.argmax(np.abs(features))
    sign = np.sign(features[idx])
    avg = np.mean(train_cwlite[:,idx])
    
    poi_candi.append([[idx,avg,sign,corr_poi],curr_poi_nano])


In [ ]:
# filter the POIs
poi_lite = []
poi_nano = []

for poi in poi_candi:
    
    poi_corr = abs(poi[0][3])
    
    if abs(poi_corr > 0.6):
        poi_lite.append(poi[0])
        poi_nano.append(poi[1])
        
print(len(poi_lite))

In [ ]:
iter_n = 2000 # the number of tests, should be fewer than half n_trace_all/4
cnt = 0
sig = 0

print(len(poi_lite))

for i in trange(iter_n):
    
    # victim trace from lite
    v_trace = test_cwlite[i*4:i*4+4]
    # template trace from nano
    t_trace = test_cwnano[i*4+1]
    
    trace_compact = []

    for ii in range(4):
        trace_compact.append(np.array([sign*(v_trace[ii][idx]-avg) for [idx,avg,sign,_] in poi_lite]))

    trace_ref_compact = np.array([t_trace[idx]-avg for [idx,avg] in poi_nano])
    
    c = np.zeros(4)

    for ii in range(4):
        c[ii] = corr(trace_compact[ii],trace_ref_compact)
        
    if c.argmax(axis=0) == 1:
        cnt += 1
        
    sig += c[1] - max(c[0] , c[2] , c[3])
        
print("accuracy:\t", cnt/iter_n)     
print("significance:\t", sig/iter_n)

In [ ]:
scope.dis()